In [ ]:
'''First Phase Environment implemented according to scratch notes from call on 12/11/20.'''

In [2]:
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import copy
import random
from scipy.stats import poisson
from scipy.stats import geom
from scipy.stats import hypergeom
from scipy.integrate import odeint

from tf_agents.environments import py_environment
from tf_agents.specs import BoundedArraySpec
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories.time_step import TimeStep

In [22]:
class Env_P1(py_environment.PyEnvironment):
    def __init__(self,
                population_herd1 = 100,
                population_herd2 = 200,
                exchanged_members = 5,
                weeks_until_exchange = 2,
                rand_recovery_prob = 0.005,
                rand_infection_prob = 0.01,
                ):
        super(Env_P1, self).__init__()
        self._state = None
        self._discount = 1.0
        self._time = 0
        self._episode_length = 0
        self._tests = []
        self._reward = 0
        self._c_tests = 1    #cost for each test
        self._c_prime_tests = 50    #organizational costs tests
        self._e_removed = 10    #individual replacement cost
        self._weeks_until_testresults = 3
        self._population_herd1 = population_herd1
        self._population_herd2 = population_herd2
        self._exchanged_members = exchanged_members    #k from scrapsheet
        self._weeks_until_exchange = weeks_until_exchange    #T from scrapsheet
        self._rand_recovery_prob = rand_recovery_prob    #g from scrapsheet
        self._rand_infection_prob = rand_infection_prob    #q from scrapsheet
    
    def action_spec(self):
        #Actions for: number of subjects to be tested h1, h2. number of subjects to be eliminated h1, h2
        return BoundedArraySpec((1,4), np.float32, minimum=0, maximum=1)
    
    
    def observation_spec(self):
        # tau, x0, x1 for both herds
        return BoundedArraySpec((1,6), np.int32, minimum=0, maximum= int('inf'))
    
    
    def _reset(self):
        self._state = np.zeros((2,6), np.int32)
        initial_infected_h1 = np.random.randint(low = 1, high = (self._population_herd1/8))
        self._time = 0
        self._reward = 0
        self._episode_length = geom.rvs(p = 1/270)
        self._state[1][3] = 0    #infected h2
        self._state[1][2] = initial_infected_h1    #infected h1
        self._state[1][1] = self._population_herd2
        self._state[1][0] = self._population_herd1
        self._state[0][5] = 0    #x1 tested pos h2
        self._state[0][4] = 0    #x0 tested neg h2
        self._state[0][3] = -1    #tau time since test h2
        self._state[0][2] = 0    #x1 tested pos h1
        self._state[0][1] = 0    #x0 tested neg h1
        self._state[0][0] = -1    #tau time since test h1
        observation = np.zeros((1,6), np.int32)
        observation[0] = self._state[0]
        return TimeStep(StepType.FIRST, reward=0,
                    discount=1.0, observation = observation)
    
    def _test(self, herd = -1, num_tests = 0):
        '''
        randomly draws num_tests subjects of a herd,
        then looks whether they are infected or not before returning results
        '''
        if herd >= 0 and num_tests > 0:
            test_out = hypergeom.rvs(M = self._state[1][herd], n = self._state[1][herd+2], N = int(num_tests), size = 1)
            testresults = np.zeros(3, np.int32)
            testresults[1] = num_tests - test_out #negative tests
            testresults[2] = test_out #positive tests
            return testresults
        else:
            return np.zeros(3, np.int32)
        
    def _transfer(self, origin_herd = -1, target_herd = -1):
        #failsafe for k>n?
        if origin_herd >= 0 and target_herd >=0 and self._time % self._weeks_until_exchange == 0:
            infected_transfers = hypergeom.rvs(M = self._state[1][origin_herd], 
                                                 n = self._state[1][origin_herd+2], N = self._exchanged_members, size = 1)
            susceptible_transfers = self._exchanged_members - infected_transfers
            return np.array([susceptible_transfers, infected_transfers])    
        else:
            return None
        
    def _model(self, action: np.ndarray, s1 = False, s2 = False):
        initial_state = self._state[1]
        #Model for one herd
        def f(x):
            s_to_i = 0
            for i in range (0, round(x[1])):
                s_to_i += poisson.rvs(0.6)
            s_to_i = round(min(s_to_i, x[0]-1))
            dsdt = x[0] - s_to_i - round(self._rand_infection_prob * x[0]) + round(self._rand_recovery_prob * x[1])
            didt = x[1] + s_to_i + round(self._rand_infection_prob * x[0]) - round(self._rand_recovery_prob * x[1])
            return np.array([dsdt, didt])
        
        #One step for each herd
        if s1:
            initial_state_h2 = [self._state[1][1]-self._state[1][3], self._state[1][3]]
            S1, I1 = [self._state[1][0], 0]
            S2, I2 = f(x = initial_state_h2)
        elif s2: 
            initial_state_h1 = [self._state[1][0]-self._state[1][2], self._state[1][2]]
            S1, I1 = f(x = initial_state_h1)
            S2, I2 = [self._state[1][1], 0]
        else:
            initial_state_h1 = [self._state[1][0]-self._state[1][2], self._state[1][2]]
            initial_state_h2 = [self._state[1][1]-self._state[1][3], self._state[1][3]]
            S1, I1 = f(x = initial_state_h1)
            S2, I2 = f(x = initial_state_h2)
        return np.array([S1+I1, S2+I2, I1, I2])
    
    def _reward_func(self, action: np.ndarray):
        tau_1 = np.round(action[0] * self._state[1][1]) 
        tau_2 = np.round(action[1] * self._state[1][2])
        indicator_1, indicator_2, s1, s2 = 0, 0, 0, 0 
        if action[2] > 1/2 and action[3] <= 1/2:
            s1 = 1
        if action[3] > 1/2 and action[2] <= 1/2:
            s2 = 1
        if tau_1 > 0:
            indicator_1 = 1
        if tau_2 > 0:
            indicator_2 = 1
        self._reward -= self._discount * (tau_1 * self._c_tests + indicator_1 * self._c_prime_tests)
        self._reward -= self._discount * (tau_2 * self._c_tests + indicator_2 * self._c_prime_tests)
        self._reward -= self._discount * (s1 * self._state[1][0] * self._e_removed + s2 * self._state[1][1] * self._e_removed)
        return self._reward
    
    def _step(self, action: np.ndarray):
        '''
        step should include model with:
        - ode's for both herds
        - transfer function
        TODOS: Check chronology, fix model, implement removal
        '''
        if self._current_time_step.is_last():
            return self._reset()
        self._time += 1
        origin_herd = 0
        target_herd = 1
        transfers = self._transfer(origin_herd = origin_herd, target_herd = target_herd)
        if transfers is not None:
            self._state[1][origin_herd] -= transfers[0] + transfers[1]
            self._state[1][target_herd] += transfers[0] + transfers[1]
            self._state[1][origin_herd+2] -= transfers[1]
            self._state[1][target_herd+2] += transfers[1]
            
        #interpreting actions
        num_test_h1 = np.round(action[0] * self._state[1][1])
        num_test_h2 = np.round(action[1] * self._state[1][2])
        
        rem_h1 = False
        rem_h2 = False
        if action[2] > 1/2 and action[3] <= 1/2:
            rem_h1 = True
        if action[3] > 1/2 and action[2] <= 1/2:
            rem_h2 = True
            
        #Model should make a step in between transfer and test
        print(self._state[1])
        self._state[1][0:4] = self._model(action, s1 = rem_h1, s2 = rem_h2)
        print(self._state[1])
        #Testing 
        self._tests.append(self._test(herd = 0, num_tests = num_test_h1))
        self._tests.append(self._test(herd = 1, num_tests = num_test_h2))
        
        for i in range (0, np.ma.size(self._tests, axis = 0)):
            if self._tests[i][0] == self._weeks_until_testresults:
                self._state[0][5] = self._tests[i+1][2]    #x1 tested pos h2
                self._state[0][4] = self._tests[i+1][1]    #x0 tested neg h2
                self._state[0][3] = self._weeks_until_testresults    
                self._state[0][2] = self._tests[i][2]    #x1 tested pos h1
                self._state[0][1] = self._tests[i][1]    #x0 tested neg h1
                self._state[0][0] = self._weeks_until_testresults
                self._tests.pop(i)
                self._tests.pop(i+1)
                break
        for i in range (0, np.ma.size(self._tests, axis = 0)):
            self._tests[i][0] += 1 
            
        #Reward function
        reward = self._reward_func(action)
        
        #output
        if self._time == self._episode_length:
            return TimeStep(StepType.LAST, reward=reward, discount=self._discount, observation=[self._state[0]])
        else:
            return TimeStep(StepType.MID, reward=reward, discount=self._discount, observation=[self._state[0]])
        

In [23]:
env = Env_P1()
env.reset()
print(env.step([0.3, 0.1, 0, 0]))

[100 200   4   0   0   0]
[100 200  11   2   0   0]
TimeStep(step_type=array(1), reward=-161.0, discount=1.0, observation=[array([-1,  0,  0, -1,  0,  0])])
